In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-products-dataset/Gaming Consoles.csv
/kaggle/input/amazon-products-dataset/Car Electronics.csv
/kaggle/input/amazon-products-dataset/Janitorial and Sanitation Supplies.csv
/kaggle/input/amazon-products-dataset/All Electronics.csv
/kaggle/input/amazon-products-dataset/All Books.csv
/kaggle/input/amazon-products-dataset/Make-up.csv
/kaggle/input/amazon-products-dataset/Travel Accessories.csv
/kaggle/input/amazon-products-dataset/Indian Language Books.csv
/kaggle/input/amazon-products-dataset/Car and Bike Care.csv
/kaggle/input/amazon-products-dataset/Sunglasses.csv
/kaggle/input/amazon-products-dataset/Bags and Luggage.csv
/kaggle/input/amazon-products-dataset/Yoga.csv
/kaggle/input/amazon-products-dataset/Sportswear.csv
/kaggle/input/amazon-products-dataset/Fiction Books.csv
/kaggle/input/amazon-products-dataset/Exam Central.csv
/kaggle/input/amazon-products-dataset/Home Storage.csv
/kaggle/input/amazon-products-dataset/Toys Gifting Store.csv
/kaggle/input/amazon-pr

# Create main dataset

In [31]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lokeshparab/amazon-products-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/amazon-products-dataset


In [32]:
import glob
import os

In [33]:
# Path to folder containing CSV files
DATA_FOLDER = "/kaggle/input/amazon-products-dataset/*.csv"

In [34]:
# Get all CSV file paths
csv_files = glob.glob(DATA_FOLDER)

print(f"Number of CSV files found: {len(csv_files)}")

Number of CSV files found: 140


In [35]:
# Read and combine
df_list = []

for file in csv_files:
    temp_df = pd.read_csv(file)
    
    temp_df["source_file"] = os.path.basename(file)
    
    df_list.append(temp_df)

In [36]:
# Concatenate all dataframes
combined_df = pd.concat(df_list, ignore_index=True)
print("Final dataset shape:", combined_df.shape)

Final dataset shape: (1103170, 11)


In [37]:
# Save dataset
combined_df.to_csv("amazon_products_main.csv", index=False)

# Data cleaning

In [38]:
df = pd.read_csv("/kaggle/working/amazon_products_main.csv")
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,source_file,Unnamed: 0
0,Electronic Spices 2.75 Inch 4ω (Ohm) 400w Max ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/61G5k3T0ff...,https://www.amazon.in/Electronic-Spices-Power-...,NaN,NaN,₹129,₹199,Car Electronics.csv,NaN
1,"ZQWINT Bluetooth Car Adapter, Mini USB Bluetoo...",car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/51NLOKwNsL...,https://www.amazon.in/ZQWINT-Bluetooth-Transmi...,NaN,NaN,₹219,₹999,Car Electronics.csv,NaN
2,PROTECTRON 6.35X32mm GLASS FUSE/INVERTER FUSE ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/71St2ruv+N...,https://www.amazon.in/PROTECTRON-6-35X32mm-GLA...,3.8,32,NaN,₹105,Car Electronics.csv,NaN
3,Cave Maruti Suzuki Male-Female Stereo Coupler ...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/31npb8UF2y...,https://www.amazon.in/Maruti-Suzuki-Stereo-Cou...,4.1,7,₹582,₹873,Car Electronics.csv,NaN
4,COVERBLACK Rubber Back Cover for Infinix X6815...,car & motorbike,Car Electronics,https://m.media-amazon.com/images/I/611zXbVxbN...,https://www.amazon.in/COVERBLACK-Infinix-X6815...,NaN,NaN,₹148,₹799,Car Electronics.csv,NaN


In [39]:
df.shape

(1103170, 11)

In [40]:
# drop unnecessary column
df = df.drop(columns=['Unnamed: 0']) 

In [41]:
# Standardize column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

In [42]:
# Remove duplicated products
df.drop_duplicates(
    subset=["name", "main_category", "sub_category"],
    inplace=True
)

print("After removing duplicates:", df.shape)

After removing duplicates: (460392, 10)


In [43]:
# Clean price column
def clean_price(col):
    return (
        col.astype(str)
           .str.replace("₹", "", regex=False)
           .str.replace(",", "", regex=False)
           .str.strip()
           .replace("nan", np.nan)
           .astype(float)
    )

df["discount_price"] = clean_price(df["discount_price"])
df["actual_price"] = clean_price(df["actual_price"])

In [44]:
# Clean rating column
df["ratings"] = (
    df["ratings"]
    .astype(str)
    .str.extract(r"(\d+\.?\d*)")[0]
    .astype(float)
)

In [45]:
# Clean number of ratings
df["no_of_ratings"] = (
    df["no_of_ratings"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+)")[0]
    .astype(float)
)

In [46]:
# Handale missing values
df.isnull().sum()

name                   0
main_category          0
sub_category           0
image                  0
link                   0
ratings           140606
no_of_ratings     137836
discount_price     49329
actual_price       12659
source_file            0
dtype: int64

In [47]:
# Drop if both prices missing
df = df.dropna(subset=["discount_price", "actual_price"], how="all")

In [48]:
# Create useful derived features
df["discount_percent"] = np.where(
    df["actual_price"] > 0,
    ((df["actual_price"] - df["discount_price"]) / df["actual_price"]) * 100,
    0
)

In [49]:
# Flag heavily discounted products
df["high_discount"] = df["discount_percent"] >= 40

In [50]:
# Fix category text
df["main_category"] = df["main_category"].str.strip().str.title()
df["sub_category"] = df["sub_category"].str.strip().str.title()

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 447733 entries, 0 to 597691
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              447733 non-null  object 
 1   main_category     447733 non-null  object 
 2   sub_category      447733 non-null  object 
 3   image             447733 non-null  object 
 4   link              447733 non-null  object 
 5   ratings           314468 non-null  float64
 6   no_of_ratings     317238 non-null  float64
 7   discount_price    411063 non-null  float64
 8   actual_price      447733 non-null  float64
 9   source_file       447733 non-null  object 
 10  discount_percent  411066 non-null  float64
 11  high_discount     447733 non-null  bool   
dtypes: bool(1), float64(5), object(6)
memory usage: 41.4+ MB


In [52]:
df.describe()

,ratings,no_of_ratings,discount_price,actual_price,discount_percent
count,314468.000000,317238.000000,4.110630e+05,4.477330e+05,411066.000000
mean,3.845126,871.487171,2.136833e+03,2.615266e+04,49.709609
std,1.044059,8898.792795,8.234613e+03,1.479564e+07,21.215472
min,1.000000,1.000000,8.000000e+00,0.000000e+00,0.000000
25%,3.500000,4.000000,3.680000e+02,8.990000e+02,35.693848
50%,3.900000,22.000000,6.250000e+02,1.499000e+03,50.550092
75%,4.300000,146.000000,1.299000e+03,2.999000e+03,66.465191
max,100.000000,589547.000000,1.249990e+06,9.900000e+09,99.999994


In [53]:
df.isnull().mean() * 100

name                 0.000000
main_category        0.000000
sub_category         0.000000
image                0.000000
link                 0.000000
ratings             29.764391
no_of_ratings       29.145719
discount_price       8.190149
actual_price         0.000000
source_file          0.000000
discount_percent     8.189479
high_discount        0.000000
dtype: float64

In [54]:
df.to_csv("amazon_products_main_cleaned.csv", index=False)